# RPA GESTIÓN DE PLATAFORMA

En este notebook se desarrolla un código que gestiona una plataforma, cambiando entre diferentes interfaces de la misma, pulsando en distintos botones en base a un listado de registros que se introduce como input.

Incluye una recolección de capturas de pantalla para dejar evidencias del proceso, así como una parte de reconcimiento óptico de caracteres (OCR) para identificar las distintas variables para la toma de decisiones y los distintos caminos a seguir por la plataforma.

El código es conceptual para ilustrar el proceso completo, no está activo. Requiere de adaptación.

In [ ]:
# Importación de librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## Carga de Datos

In [ ]:

ruta_ppal = '/Users/Documents/TRABAJO/AI4G/REPOSITORIO CASOS DE USO/AUTOMATISMOS/RPA_GESTIÓN_PLATAFORMA/'
ruta_archivo = '/Users/Documents/TRABAJO/AI4G/REPOSITORIO CASOS DE USO/AUTOMATISMOS/RPA_GESTIÓN_PLATAFORMA/datos.csv'

# Carga del dataset
try:
    df = pd.read_csv(ruta_archivo)
    print("Datos cargados correctamente.")
except FileNotFoundError:
    print("Archivo no encontrado. Verifica la ruta.")

## RECORTE_OCR

In [ ]:
from PIL import Image
import os

# Rutas de los directorios
directorio_origen = r"ruta_ppal\capturas_evs_cierres\captura_3_imagen"
directorio_destino = r"ruta_ppal\capturas_evs_cierres\captura_imagen_3_rec"

# Dimensiones del recorte
ancho_recorte = 85  # Ancho del rectángulo de recorte
alto_recorte = 40  # Alto del rectángulo de recorte

# Punto de inicio del recorte (como porcentaje del ancho y alto de la imagen)
inicio_x = 0.755  # 40% del ancho de la imagen
inicio_y = 0.33  # 50% del alto de la imagen

# Asegurarse de que el directorio de destino existe
if not os.path.exists(directorio_destino):
    os.makedirs(directorio_destino)

# Procesar cada archivo de imagen en el directorio
for filename in os.listdir(directorio_origen):
    print(filename)
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Ajusta según tus tipos de archivo
        ruta_completa = os.path.join(directorio_origen, filename)
        with Image.open(ruta_completa) as img:
            ancho, alto = img.size

            # Calcula el punto de inicio del recorte
            left = int(ancho * inicio_x)
            top = int(alto * inicio_y)

            # Asegúrate de que el recorte no exceda las dimensiones de la imagen
            right = min(left + ancho_recorte, ancho)
            bottom = min(top + alto_recorte, alto)

            img_recortada = img.crop((left, top, right, bottom))
            img_recortada.save(os.path.join(directorio_destino, "recortada_" + filename))


#RECONOCIMIENTO CARACTERES
import pytesseract
from PIL import Image
import os
import pandas as pd


pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


directorio_destino = r"ruta_ppal\capturas_evs_cierres\captura_imagen_3_rec"

# directorio_destino = r"ruta_ppal\prueba_"

# Lista para almacenar los resultados
resultados = []

# Procesar cada archivo de imagen en el directorio
i=0
for filename in os.listdir(directorio_destino):
    i+=1
    print(i)
    print(filename)
    if filename.endswith(".jpg") or filename.endswith(".png"):
        ruta_completa = os.path.join(directorio_destino, filename)
        with Image.open(ruta_completa) as img:
            # Uso de Tesseract para reconocer texto en la imagen
            texto = pytesseract.image_to_string(img, config='--psm 6').replace(" ", "").replace("\n", "")
            print(texto)
            resultados.append({'Nombre del Archivo': filename, 'Texto': texto})

# Convertir la lista de resultados en un DataFrame
df = pd.DataFrame(resultados)

df['Texto'] = df['Texto'].astype(str)

# Crear un nuevo DataFrame con los nombres de las imágenes cuyo texto no sea "cer"
# df_no_cer = df.loc[df['Texto'].str.replace(" ", "").replace("\n", "").str.lower() != 'cer']
df_no_cer = df.loc[~df['Texto'].str.replace(" ", "").replace("\n", "").str.lower().str.contains('cerr(ado)?')]


df_no_cer['Codigo'] = df_no_cer['Nombre del Archivo'].str.extract(r'recortada_(\d+)_\d+\.png')

df_codigos = df_no_cer[['Codigo']]


df_codigos.to_excel(r'ruta_ppal\prueba_\codigos.xlsx', index=False)


## RPA_plataforma.py

In [ ]:

import os #Necesaria para conseguir la ruta de un archivo
from datetime import datetime #Necesaria para el tratamiento de fechas
# from selenium import webdriver
import time
import pandas as pd #Necesaria para el tratamiento de datos
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# import requests
# from selenium.common.exceptions import ElementClickInterceptedException
# from bs4 import BeautifulSoup
import warnings #Para desactivar los warnings
# from selenium.webdriver.support.ui import Select
from datetime import datetime, timedelta

import pyautogui
from PIL import Image
import pytesseract
# import requests

warnings.filterwarnings("ignore") #Desactivo los warnings. Poner solo cuando el codigo esta depurado
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.reset_option('display.max_colwidth')
pd.set_option('display.width', 600)  # Ajusta el ancho a 200 caracteres
print("Libreri­as importadas")

def detecta_loading (ruta):
    captura = pyautogui.screenshot()
    captura.save(ruta)
    # Dimensiones del recorte
    ancho_recorte = 200  # Ancho del rectángulo de recorte
    alto_recorte = 50  # Alto del rectángulo de recorte

    # Punto de inicio del recorte (como porcentaje del ancho y alto de la imagen)
    inicio_x = 0.47  # 50% del ancho de la imagen
    inicio_y = 0.495  # 50% del alto de la imagen
    with Image.open(f'{ruta}') as img:
        ancho, alto = img.size

        # Calcula el punto de inicio del recorte
        left = int(ancho * inicio_x)
        top = int(alto * inicio_y)

        # Asegúrate de que el recorte no exceda las dimensiones de la imagen
        right = min(left + ancho_recorte, ancho)
        bottom = min(top + alto_recorte, alto)

        img_recortada = img.crop((left, top, right, bottom))
        # Extraer el nombre base y la extensión del archivo
        base_dir, nombre_archivo = os.path.split(ruta)
        nombre_base, extension = os.path.splitext(nombre_archivo)
        
        # Crear un nuevo nombre de archivo para la imagen recortada
        nombre_recortado = f"recorte_{nombre_base}{extension}"
        ruta_recortada = os.path.join(base_dir, nombre_recortado)
        time.sleep(0.2)
        # Guardar la imagen recortada
        img_recortada.save(ruta_recortada)
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
        time.sleep(0.2)
        with Image.open(ruta_recortada) as img:
            # Uso de Tesseract para reconocer texto en la imagen
            texto = pytesseract.image_to_string(img, config='--psm 6').replace(" ", "").replace("\n", "")
            # print(texto)
        
        return texto
    
def detecta_error (ruta):
    captura = pyautogui.screenshot()
    captura.save(ruta)
    # Dimensiones del recorte
    # Dimensiones del recorte
    ancho_recorte = 85  # Ancho del rectángulo de recorte
    alto_recorte = 40  # Alto del rectángulo de recorte
    
    # Punto de inicio del recorte (como porcentaje del ancho y alto de la imagen)
    inicio_x = 0.755  # 40% del ancho de la imagen
    inicio_y = 0.33  # 50% del alto de la imagen
    
    with Image.open(f'{ruta}') as img:
        ancho, alto = img.size

        # Calcula el punto de inicio del recorte
        left = int(ancho * inicio_x)
        top = int(alto * inicio_y)

        # Asegúrate de que el recorte no exceda las dimensiones de la imagen
        right = min(left + ancho_recorte, ancho)
        bottom = min(top + alto_recorte, alto)

        img_recortada = img.crop((left, top, right, bottom))
        # Extraer el nombre base y la extensión del archivo
        base_dir, nombre_archivo = os.path.split(ruta)
        nombre_base, extension = os.path.splitext(nombre_archivo)
        
        # Crear un nuevo nombre de archivo para la imagen recortada
        nombre_recortado = f"recorte_{nombre_base}{extension}"
        ruta_recortada = os.path.join(base_dir, nombre_recortado)
        time.sleep(0.2)
        # Guardar la imagen recortada
        img_recortada.save(ruta_recortada)
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
        time.sleep(0.2)
        with Image.open(ruta_recortada) as img:
            # Uso de Tesseract para reconocer texto en la imagen
            texto = pytesseract.image_to_string(img, config='--psm 6').replace(" ", "").replace("\n", "")
            # print(texto)
        
        return texto

def detecta_error_web (ruta):
    captura = pyautogui.screenshot()
    captura.save(ruta)
    # Dimensiones del recorte
    # Dimensiones del recorte
    ancho_recorte = 80  # Ancho del rectángulo de recorte
    alto_recorte = 40  # Alto del rectángulo de recorte
    
    # Punto de inicio del recorte (como porcentaje del ancho y alto de la imagen)
    inicio_x = 0.105  # 40% del ancho de la imagen
    inicio_y = 0.3  # 50% del alto de la imagen
    
    with Image.open(f'{ruta}') as img:
        ancho, alto = img.size

        # Calcula el punto de inicio del recorte
        left = int(ancho * inicio_x)
        top = int(alto * inicio_y)

        # Asegúrate de que el recorte no exceda las dimensiones de la imagen
        right = min(left + ancho_recorte, ancho)
        bottom = min(top + alto_recorte, alto)

        img_recortada = img.crop((left, top, right, bottom))
        # Extraer el nombre base y la extensión del archivo
        base_dir, nombre_archivo = os.path.split(ruta)
        nombre_base, extension = os.path.splitext(nombre_archivo)
        
        # Crear un nuevo nombre de archivo para la imagen recortada
        nombre_recortado = f"recorte_{nombre_base}{extension}"
        ruta_recortada = os.path.join(base_dir, nombre_recortado)
        time.sleep(0.2)
        # Guardar la imagen recortada
        img_recortada.save(ruta_recortada)
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
        time.sleep(0.2)
        with Image.open(ruta_recortada) as img:
            # Uso de Tesseract para reconocer texto en la imagen
            texto = pytesseract.image_to_string(img, config='--psm 6').replace(" ", "").replace("\n", "")
            # print(texto)
        
        return texto

start_time_nat = time.time()  # guarda el tiempo actual
#SETTINGS
area='geografia'
hoy = datetime.now().strftime('%d_%m_%Y')

#Ruta Aníbal
ruta_importacion_excel = r"ruta_ppal\FICHEROS__S\datos.xlsx"
#Ruta Miguel
ruta_importacion_excel = r"ruta_ppal\FICHEROS__S\datos.xlsx"
ruta_importacion_excel = r"ruta_ppal\Pasar_a_cerrado.xlsx"

df__s = pd.read_excel(ruta_importacion_excel,sheet_name='Hoja1',dtype=str)

df__s.rename(columns={df__s.columns[0]: '_S'}, inplace=True)

df__s_invertido = df__s.iloc[::-1]
df__s=df__s_invertido.copy()

# df__s=df__s.head(10)
# df__s = df__s.iloc[10:20]
print("Datos importados")

###APERTURA MANUAL DE geS EN cx. HAY QUE PONER LA VENTANA EN MODO PANTALLA COMPLETA. 
###HECHO PARA 3 PANTALLAS Y EL ORDENADOR A LA IZQUIERDA, COMPROBAR CON OTRAS MODIFICACIONES DE LAS POSICIONES DE LAS PANTALLAS
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"Inicio a las {formatted_time}")
#Seleccion perfil usuario CDS
pyautogui.click(316,222,button='left')
time.sleep(1)
pyautogui.click(294,260,button='left')
time.sleep(1)
#Seleccion territorio
pyautogui.click(368,258,button='left')
time.sleep(1)
if area == 'geografia':
    pyautogui.click(338,298,button='left')
elif area == 'BALEARES':
    pyautogui.click(338,317,button='left')
elif area == 'CANARIAS':
    pyautogui.click(338,338,button='left')
elif area == 'CATALUNYA':
    pyautogui.click(338,362,button='left')
elif area == 'ARAGON':
    pyautogui.click(338,382,button='left')
else:
    print("Area no seleccionada")
time.sleep(1)
#Pulsar botón Perfil geS
pyautogui.click(343,331,button='left')
time.sleep(3) #Más tiempo por cambio de página


_s_ejecutados = pd.DataFrame(columns=['_'])
_s_error = pd.DataFrame(columns=['_'])
# Eliminar los últimos 75 registros
# copia_seg=_s_ejecutados.copy()

# _s_ejecutados=copia_seg.copy()

# _s_ejecutados = _s_ejecutados.iloc[:-75]

df__s = df__s[~df__s['_S'].isin(_s_ejecutados['_'])]

ruta_loading="ruta_ppal/capturas__s_cierres/LOADING/load.png"
ruta_estado_cerrado="ruta_ppal/capturas__s_cierres/LOADING/cerrado.png"


_s_error.rename(columns={_s_error.columns[0]: '_S'}, inplace=True)
df__s=_s_error.copy()

print(area)
i=0
start_time = datetime.now()
end_time = datetime.now()

for index, fila in df__s.iterrows():
    _ = fila['_S']
    # _ = '15241101'
    elapsed_time = (end_time - current_time).total_seconds()
    
    current_time = datetime.now()
    print(f"Ha tardado {elapsed_time}")

    # Estima el tiempo restante
    events_left = len(df__s) - (i + 1)
    estimated_time_remaining = elapsed_time * events_left
    
    # Calcula la hora prevista de finalización
    estimated_end_time = current_time + timedelta(seconds=estimated_time_remaining)
    
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    estimated_end_formatted = estimated_end_time.strftime("%Y-%m-%d %H:%M:%S")
    # Formatear el tiempo estimado restante en horas y minutos
    hours_remaining = int(estimated_time_remaining // 3600)
    minutes_remaining = int((estimated_time_remaining % 3600) // 60)
   
    print(f'{_}--{i + 1} de {len(df__s)}--faltan {hours_remaining} horas y {minutes_remaining} minutos. Fin a las {estimated_end_formatted}')
    #Pulsar botón Gestión de _s
    pyautogui.click(595,317,button='left')

    time.sleep(0.5)
    
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.5)

    estado_web=detecta_error_web(ruta_estado_cerrado)
    print(estado_web)
    
    if "temporal" not in estado_web:
        nuevo___df = pd.DataFrame({'_': [_]})
        print(f"Error en el _ {_}, se pasa al siguiente")
        _s_error = pd.concat([_s_error, nuevo___df], ignore_index=True)
        #Pulsar en Inicio
        pyautogui.click(57,207,button='left')
        esperar = True
        while esperar:
            estado=detecta_loading(ruta_loading)
            if estado != "Loading":
                esperar = False
            else:
                print(estado)
            time.sleep(0.5)
        time.sleep(1)
        continue


    pyautogui.click(405,407,button='left')
    pyautogui.click(405,407,button='left')
    time.sleep(1)
    #Escribir event_cod
    captura = pyautogui.screenshot()
    captura.save(f"ruta_ppal/capturas__s_cierres/{_}.png")
    pyautogui.click(405,407,button='left')
    pyautogui.write(f'{_}')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
        
    captura1 = pyautogui.screenshot()
    captura1.save(f"ruta_ppal/capturas__s_cierres/{_}_1.png")

    #Pulsar en Filtrar
    pyautogui.click(1288,264,button='left')

    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)

    #Pulsar en Registro
    pyautogui.click(850,376,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
    #Pulsar en Cerrar
    pyautogui.click(1079,265,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
    #Pulsar en Confirmar
    pyautogui.click(1436,310,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
    #Pulsar en Cerrar dialog
    pyautogui.click(1570,299,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
    #Pulsar en Guardar
    pyautogui.click(1890,413,button='left')
    
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.2)
    
    pyautogui.click(1890,413,button='left')
    captura2 = pyautogui.screenshot()
    captura2.save(f"ruta_ppal/capturas__s_cierres/{_}_2.png")
    captura2.save(f"ruta_ppal/capturas__s_cierres/captura_3_imagen/{_}_2.png")
    time.sleep(1)
        
    time.sleep(0.5)
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.3)
    
    estado_final=detecta_error(ruta_estado_cerrado)
    print(estado_final)
    
    nuevo___df = pd.DataFrame({'_': [_]})
    
    if "Cerrado" not in estado_final:
        print(f"Error en el _ {_}, se pasa al siguiente")
        _s_error = pd.concat([_s_error, nuevo___df], ignore_index=True)
        #Pulsar en Inicio
        pyautogui.click(57,207,button='left')
        esperar = True
        while esperar:
            estado=detecta_loading(ruta_loading)
            if estado != "Loading":
                esperar = False
            else:
                print(estado)
            time.sleep(0.5)
        time.sleep(1)
        continue
    
    #Pulsar en Inicio
    pyautogui.click(57,207,button='left')
    esperar = True
    while esperar:
        estado=detecta_loading(ruta_loading)
        if estado != "Loading":
            esperar = False
        else:
            print(estado)
        time.sleep(0.5)

    i+=1
    _s_ejecutados = pd.concat([_s_ejecutados, nuevo___df], ignore_index=True)

    end_time = datetime.now()

print("Proceso completado.")


